In [82]:
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [83]:
# Store csv created in part one into a DataFrame
cities_weather = pd.read_csv("city_weather.csv")
cities_weather = cities_weather.rename(columns={"Unnamed: 0": "City_ID"})
cities_weather.head()

,City_ID,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
0,0,Rikitea,-23.1203,-134.9692,79.20,74.0,5.0,21.18,PF,1.616820e+09
1,1,Atuona,-9.8000,-139.0333,80.91,77.0,56.0,18.57,PF,1.616820e+09
2,2,Hayden,47.7660,-116.7866,46.40,61.0,90.0,6.91,US,1.616820e+09
3,3,Busselton,-33.6500,115.3333,75.00,53.0,20.0,3.00,AU,1.616820e+09
4,4,Tigil',57.8000,158.6667,19.47,80.0,90.0,11.88,RU,1.616820e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [84]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_weather[['latitude', 'longitude']].astype(float)
humidity = cities_weather['humidity'].astype(float)

fig = gmaps.figure()

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

# Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
# Narrow down cities that fit criteria of:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_weather = cities_weather.loc[(cities_weather['max temp'] > 70.00) & (cities_weather['max temp'] < 90.00) & 
                                   (cities_weather['wind speed'] < 10) & (cities_weather['cloudiness'] == 0)]

ideal_weather.head()

,City_ID,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
6,6,Riyadh,24.6877,46.7219,77.00,19.0,0.0,4.61,SA,1.616820e+09
33,35,Bakhtiyārpur,25.4667,85.5167,80.60,28.0,0.0,5.75,IN,1.616820e+09
51,53,Patrocínio,-18.9439,-46.9925,73.40,68.0,0.0,1.14,BR,1.616820e+09
73,78,Nioro,13.3500,-15.7500,75.18,34.0,0.0,9.13,GM,1.616820e+09
80,87,Arraial do Cabo,-22.9661,-42.0278,77.00,83.0,0.0,9.22,BR,1.616820e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [130]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather[['city_name', 'country', 'latitude', 'longitude']]
hotel_df['Hotel Name'] = ""
hotel_df.rename(columns={"city_name": "City"})
hotel_df.head()

<ipython-input-130-0d5a28fd63ed>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,city_name,country,latitude,longitude,Hotel Name
6,Riyadh,SA,24.6877,46.7219,
33,Bakhtiyārpur,IN,25.4667,85.5167,
51,Patrocínio,BR,-18.9439,-46.9925,
73,Nioro,GM,13.3500,-15.7500,
80,Arraial do Cabo,BR,-22.9661,-42.0278,


In [147]:
# Set parameters to search for a hotel
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"location": (lat,lng),
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = hotel_df['latitude']
    lng = hotel_df['longitude']
    params['location'] = (lat, lng)

    # make request and print url and convert to json
    response = requests.get(url, params=params).json()
    
    print(response)

    # Grab the first hotel from the results and store the name

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
{'html_attributions'

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))